Loading Tweets

In [ ]:
import sqlite3

conn = sqlite3.connect('tweet_path.db')
query = f"SELECT text FROM weather_tweets"

df_cond = pd.read_sql_query(query, conn)
conn.close()

Running CIDER for Sentiment

In [ ]:
from ciderpolarity import CIDER
cdr = CIDER(list(df_cond.text),'/output/path/',verbose=True)
cdr.fit()

Creating a temperature linguistic scale

In [ ]:
col = 'max_temperature'

n_rows = len(df_cond)

# Coldest tweets (lowest percentiles)
low_txt1 = df_cond.sort_values(col).iloc[:int(n_rows * 1 / 100)].text.tolist()  # Coldest 1%
low_txt2 = df_cond.sort_values(col).iloc[int(n_rows * 1 / 100):int(n_rows * 2 / 100)].text.tolist()  # Coldest 1-2%
low_txt3 = df_cond.sort_values(col).iloc[int(n_rows * 2 / 100):int(n_rows * 3 / 100)].text.tolist()  # Coldest 2-3%

# Hottest tweets (highest percentiles)
high_txt1 = df_cond.sort_values(col).iloc[-int(n_rows * 1 / 100):].text.tolist()  # Hottest 1%
high_txt2 = df_cond.sort_values(col).iloc[-int(n_rows * 2 / 100):-int(n_rows * 1 / 100)].text.tolist()  # Hottest 1-2%
high_txt3 = df_cond.sort_values(col).iloc[-int(n_rows * 3 / 100):-int(n_rows * 2 / 100)].text.tolist()  # Hottest 2-3%

# Middle range (neither too cold nor too hot)
middle_txt = df_cond.sort_values(col).iloc[int(n_rows * 3 / 100):-int(n_rows * 3 / 100)].clean_text.tolist()

# Add labels to tweets
low_txt1 = [f"{text} low1" for text in low_txt1]
low_txt2 = [f"{text} low2" for text in low_txt2]
low_txt3 = [f"{text} low3" for text in low_txt3]

high_txt1 = [f"{text} top1" for text in high_txt1]
high_txt2 = [f"{text} top2" for text in high_txt2]
high_txt3 = [f"{text} top3" for text in high_txt3]

# Combine labelled tweets
low_txt = low_txt1 + low_txt2 + low_txt3
high_txt = high_txt1 + high_txt2 + high_txt3

# Define seed weights
top_seeds = {"top1": 3, "top2": 2, "top3": 1}
low_seeds = {"low1": 3, "low2": 2, "low3": 1}

# Combine all text for analysis
texts = low_txt + high_txt + middle_txt

# Run CIDER
cdr = CIDER(texts, '/output/path/', pos_seeds=top_seeds, neg_seeds=low_seeds)
cdr.fit()
